In [18]:
import requests
from bs4 import BeautifulSoup

In [19]:
# fetch a web page
r = requests.get("https://www.francetvinfo.fr/sports/")
print(r.text[:1000])

<!DOCTYPE html><!--[if lt IE 7]><html class="no-js ie ie6 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 7]><html class="no-js ie ie7 lte9 lte8 lte7" lang="fr"> <![endif]--><!--[if IE 8]><html class="no-js ie ie8 lte9 lte8" lang="fr"> <![endif]--><!--[if IE 9]><html class="no-js ie ie9 lte9" lang="fr"> <![endif]--><!--[if gt IE 9]><html class="no-js ie" lang="fr"> <![endif]--><!--[if !IE]><!--><html class="no-js" lang="fr"> <!--<![endif]--><head><title>Sports</title><meta http-equiv="pragma" content="no-cache" /><meta http-equiv="expires" content="Tue, 22 Feb 2000 12:14:57 GMT" /><meta http-equiv="refresh" content="1799" /><meta property="og:title" content="Sports" /><meta property="og:description" content="Suivez en direct toute l'actualité &quot;Sports&quot; : vivez l'info en live, en images et en vidéos. Participez, commentez et partager avec Franceinfo en temps réel !" /><meta property="og:type" content="website" /><meta property="og:locale:alternate" content="fr_FR" /><meta pro

In [20]:
soup = BeautifulSoup(r.text, "html5lib")
print(soup.get_text()[:1000])

Sports  var myHash='',queryString=document.location.search;var xtref=document.referrer.replace(/[<>]/g,'').replace(/&/g,'$');myHash='xtref='+((''!=xtref)?xtref:'acc_dir');if(document.location.search&&(new RegExp("xtor=([^&]+)")).test(document.location.search)){var xtor=(new RegExp("xtor=([^&]+)")).exec(document.location.search);if(xtor){myHash=((myHash!='')?'&':'')+'xtor='+xtor[1]} console.log('document.location.search',document.location.search);var queryString=document.location.search.replace(xtor[0],'');console.log("queryString",queryString);} myHash=(document.location.hash!='')?document.location.hash+((myHash!='')?'&'+myHash:''):'#'+myHash;(function(mobileUri){var isIpad=navigator.userAgent.match(/iPad/i)!=null,isMobile="undefined"!==typeof window.screen&&"undefined"!==typeof window.screen.width&&(768>=window.screen.width);if(isMobile&&!isIpad){window.location.href="//"+window.location.hostname.replace("www","mobile")+mobileUri}})(("/sports/"+queryString+myHash));  (function(){("und

In [21]:
# extract title
summaries = soup.find_all('div', class_="flowItem")

In [22]:
summaries[0]

<div class="flowItem feature" entity="CMS-Article_3819085"><a href="/sports/rugby/tournoi-des-six-nations/tournoi-des-six-nations-le-xv-de-france-bat-l-italie-35-22-et-enchaine-une-deuxieme-victoire_3819085.html"><figure id="image_3621731"><img alt="Les Bleus Demba Bamba (à gauche) et Charles Ollivon (à droite) célèbrent leur victoire face à l\'Italie, au Stade de France, le 9 février 2020.&amp;nbsp;" botag="CMS-IMAGE_3621731" class="b-lazy" data-src="/image/75ruix182-e778/840/472/20929893.jpg" height="472px" src="data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEKAAEALAAAAAABAAEAAAICTAEAOw==" title="Les Bleus Demba Bamba (à gauche) et Charles Ollivon (à droite) célèbrent leur victoire face à l'Italie, au Stade de France, le 9 février 2020.  | PHILIPPE LOPEZ / AFP" width="840px"/></figure><div class="right-content"><h2 class="title" field="title">Le XV de France enchaîne face à l'Italie et prend les commandes du Tournoi des six nations</h2><div class="description" field="description">Après 

In [29]:
summaries[0].select_one("h2").get_text()

"Le XV de France enchaîne face à l'Italie et prend les commandes du Tournoi des six nations"

In [30]:
len(summaries)

30

In [37]:
sport_news = [x.select_one("h2").get_text() for x in summaries]

In [38]:
sport_news

["Le XV de France enchaîne face à l'Italie et prend les commandes du Tournoi des six nations",
 'DIRECT. Tournoi des six nations : un cinquième essai et les Bleus filent vers un succès serein. Regardez et commentez France-Italie',
 'Judo : le double champion olympique Teddy Riner battu pour la première fois depuis plus de neuf ans',
 'Le rendez-vous du Particulier. Contester une amende, regagner des points de permis, choisir un bon avocat',
 "Saut à la perche : avec un saut à 6,17 m, le Suédois Armand Duplantis s'empare du record du monde de Renaud Lavillenie",
 '"On rigole, on s\'entend tous très bien" : le plaisir retrouvé de jouer ensemble, clef du succès pour le XV de France ?',
 'Patinage : Didier Gailhaguet cède à la pression et démissionne',
 "En route vers Paris 2024. L'intelligence de notre corps",
 '"C\'est troublant, on se dit que ce n\'est que le commencement" : après la démission de Didier Gailhaguet, le monde du sport veut poursuivre son examen de conscience',
 'Les Bleue

In [36]:
i

0